# Cross Correlation/Matched filter tutorial

This tutorial will show how to use the matched filter capabilities of PyKlip. 


## Preliminary

### Parallelized Performance (Use of mkl)
Refer to http://pyklip.readthedocs.io/en/latest/install.html#note-on-parallelized-performance for use of mkl.

In [ ]:
try:
    import mkl
    mkl.set_num_threads(1)
except:
    print("Your code migth run slowly.")
    print("/!\ Please Read http://pyklip.readthedocs.io/en/latest/install.html#note-on-parallelized-performance")
    

### Klip POst Processing (KPOP)
The matched filter functionality are part of PyKlip's subpackage KPOP, referring to Klip POst Processing.

The KLIP POst Processing framework has been implemented with surveys application in mind. It includes several wrapper functions, which all ihnerit from the same object, and performing different tasks like matched filters, SNR map calculation, detection of high SNR blobs and others. A lot features have been automated and are therefore hidden from the user, which is appreciable when running the same sequence of tasks and many datasets.

### Speckle subtraction
The matched filter should be performed on speckle subtracted dataset so let's reduce a test dataset with PyKlip using http://pyklip.readthedocs.io/en/latest/klip_gpi.html.
This step can be skipped if reduced images are already available.

In [ ]:
import os
import glob
import pyklip.instruments.GPI as GPI
import pyklip.parallelized as parallelized

pykliproot = os.path.dirname(os.path.realpath(parallelized.__file__))
inputDir = os.path.join(pykliproot,"..","tests","data")
outputDir = inputDir

Read dataset and run speckle subtraction:

In [ ]:
# Read the datacubes using the GPIData object
filelist = glob.glob(os.path.join(inputDir,"*spdc_distorcorr.fits"))
dataset = GPI.GPIData(filelist,highpass=True,meas_satspot_flux=False,numthreads=None)

parallelized.klip_dataset(dataset, outputdir=outputDir, fileprefix="bet_Pic_test",
                          annuli=9, subsections=4, movement=1, numbasis=[1,20,50,100],
                          calibrate_flux=True, mode="ADI+SDI")

## Cross Correlation

The cross correlation is the most basic type of matched filter. It assumes that the noise is identically distributed for all the pixels in the image. See discussion in Ruffio et al. (2017).

The following shows different cases for applying the cross correlation.

### Single cube. Independent slices.

For example, pyklip produces fits cube where each slice is the a collapsed spectral cube but using different number of KL modes (filename suffix is KLmodes-all). CrossCorr will process each slice independently.

The method check_existence() allows to check for the existence of the processed file. You might want to skip it if it already exists.

In [ ]:
from pyklip.kpp.metrics.crossCorr import CrossCorr

# Definition of the cross correlation object
filename = "bet_Pic_test-KLmodes-all.fits"
read_func = lambda file_list:GPI.GPIData(file_list,recalc_centers=False,recalc_wvs=False,highpass=False)
cc_obj = CrossCorr(read_func,filename,kernel_type="gaussian",kernel_para=1.0,label="pyklip")

cc_obj.initialize(inputDir=inputDir,
                   outputDir=outputDir)
#check if the processed data exists
print("Does the processed file exist? {0}".format(cc_obj.check_existence()))
# Run cross correlation
cc_image = cc_obj.calculate()
# Save data to #pykliproot#/../tests/data/kpop_pyklip/default_out/bet_Pic_test-KLmodes-all-crossCorrgaussian.fits
cc_obj.save()
#check if the processed data exists
print("Does the processed file exist? {0}".format(cc_obj.check_existence()))

%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(cc_image[2,::-1,:])

### Single Spectral Cube

For example, pyklip produces spectral cube with the suffix "speccube". CrossCorr can first collapse the cube with a given spectrum before running the cross correlation using the two keywords "collapse=True,spectrum=spectrum". Here we are using the atmospheric models from Marley et al. (2017, in prep.) and Saumon et al. 2012.

In [ ]:
from pyklip.kpp.metrics.crossCorr import CrossCorr

# Define the spectrum from its name
spectrum = "t1000g100nc"
# Spectrum can also be user defined as commented in the line below.
#spectrum = np.ones(37)
folderName = "t1000g100nc"

# Definition of the cross correlation object
filename = "bet_Pic_test-KL20-speccube.fits"
read_func = lambda file_list:GPI.GPIData(file_list,recalc_centers=False,recalc_wvs=False,highpass=False)
cc_obj = CrossCorr(read_func,filename,kernel_type="gaussian",kernel_para=1.0,label="pyklip",
                   collapse=True,spectrum=spectrum,folderName=folderName)

cc_obj.initialize(inputDir=inputDir,
                   outputDir=outputDir)
cc_image = cc_obj.calculate()
cc_obj.save()

import matplotlib.pyplot as plt
plt.imshow(cc_image[::-1,:])

### SNR calculation

StatPerPix can be used to estimate the SNR maps. It estimate the standard deviation for each pixel in an annulus at the same separation from which the surroundings of the pixel has been masked.

Note that by not specifying the outputDir in the SNR calculation, the label and the folderName is inferred from the cross correlation map path.

In [ ]:
from pyklip.kpp.stat.statPerPix import StatPerPix

# Definition of the SNR object
read_func = lambda file_list:GPI.GPIData(file_list,recalc_centers=False,recalc_wvs=False,highpass=False)
filename = os.path.join("kpop_pyklip","t1000g100nc","bet_Pic_test-KL20-speccube-crossCorrgaussian.fits")
snr_obj = StatPerPix(read_func,filename,type="SNR")

snr_obj.initialize(inputDir=inputDir)
snr_image = snr_obj.calculate()
snr_obj.save()

import numpy as np
print("The SNR peak in the image is {0:.2f} sigma.".format(np.nanmax(snr_image)))
import matplotlib.pyplot as plt
plt.imshow(snr_image[::-1,:])

### Multiple files with different reduction spectra

KPOP has been optimized to ease the reduction of survey data. Reducing one or several files requires the same number of lines of code. The function automating the process is called kppPerDir and take a list of objects (ie tasks to be executed) and a list of spectrum over which to iterate. Each object knows if it has to iterate over the spectra or not. For example the SNR calculation will be insensitive to spectrum_list.

kppPerDir will automatically iterate over all the files matching the filename description (using wildcards).

If overwrite kept False, kppPerDir automatically verifies that the files has not already been processed.

In [2]:
import numpy  as np
spectrum_list = [np.ones(37),"t1300g100f2"]

read_func = lambda file_list:GPI.GPIData(file_list,recalc_centers=False,recalc_wvs=False,highpass=False)

# Definition of the cross correlation object
from pyklip.kpp.metrics.crossCorr import CrossCorr
filename = "bet_Pic_test-KL*-speccube.fits"
cc_obj = CrossCorr(read_func,filename,kernel_type="gaussian",kernel_para=1.0,label="pyklip",collapse=True)

# Definition of the SNR object
from pyklip.kpp.stat.statPerPix import StatPerPix
filename = os.path.join("kpop_pyklip","*","bet_Pic_test-KL*-speccube-crossCorrgaussian.fits")
snr_obj = StatPerPix(read_func,filename,type="SNR")

# Process all matching files
from pyklip.kpp.kpop_wrapper import kpop_wrapper
err_list = kpop_wrapper(inputDir,[cc_obj,snr_obj],spectrum_list=spectrum_list,mute_error = False)

# One can check if reductions have failed here
print("Printing error report:")
for err_str in err_list:
    print(err_str)

~~ INITializing CrossCorr ~~
Reading File: /home/sda/jruffio/pyklip/tests/data/bet_Pic_test-KL100-speccube.fits
Generate gaussian PSF
~~ UPDATE Spectrum CrossCorr ~~
('coucou', array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]))
Output already exist: /home/sda/jruffio/pyklip/tests/data/kpop_pyklip/custom//bet_Pic_test-KL100-speccube-crossCorrgaussian.fits
~~ UPDATE Spectrum CrossCorr ~~
('coucou', 't1300g100f2')
Spectrum model: /home/sda/jruffio/pyklip/pyklip/spectra/cloudy/t1300g100f2.flx
Output already exist: /home/sda/jruffio/pyklip/tests/data/kpop_pyklip/t1300g100f2//bet_Pic_test-KL100-speccube-crossCorrgaussian.fits
~~ INITializing CrossCorr ~~
Reading File: /home/sda/jruffio/pyklip/tests/data/bet_Pic_test-KL1-speccube.fits
Generate gaussian PSF
~~ UPDATE Spectrum CrossCorr ~~
('coucou', 't1300g100f2')
Spectrum model: /home/sda

## Matched Filter

coucou

In [ ]:
if 1:
    try:
        import mkl
        mkl.set_num_threads(1)
    except:
        print("Your code migth run slowly without mkl.")
        print("/!\ Please Read http://pyklip.readthedocs.io/en/latest/install.html#note-on-parallelized-performance")
    import os
    import glob
    import pyklip.instruments.GPI as GPI
    import pyklip.parallelized as parallelized

    pykliproot = os.path.dirname(os.path.realpath(parallelized.__file__))
    inputDir = os.path.join(pykliproot,"..","tests","data")
    outputDir = inputDir
    
    %matplotlib inline

### Run FMMF
FMMF is meant to be ran on 16 (or above) cores machine with 60GB (or above) of RAM. Even with such computers, the code will take many hours to days to run on a normal dataset. Therefore, an iPython notebook is not an optimal mean to run this code. One should consider running a script with the linux command *nice* to let it run in the background. The ideal environment for this FMMF implementation is a supercluster where several dataset can be ran in parallel.

In [ ]:
inputDir = "../tests/data/"
outputDir = inputDir

FMMFObj.initialize(inputDir=inputDir,
                   outputDir=outputDir,
                   spectrum = "t1800g100nc",
                   compact_date="myDataDate")
FMMFObj.calculate()
FMMFObj.save()

## Advanced Features

**This section is in progress.**

Most of them are automatically treated by the FMMF object but one could want to take a closer look:
Sectors Definition, spectrum, Overlap, PSF cube, spectral type of the target, fakes management, noFM

Also all kppPerDir related features when reducing campaign data.